In [1]:
import pandas as pd
import numpy as np
from time import sleep

In [2]:
not_hot_songs = pd.read_csv("not_hot_drop.csv")

In [3]:
hot_songs = pd.read_csv("top_100_hot.csv")

In [4]:
not_hot_songs.drop(columns = 'Unnamed: 0', inplace = True)
not_hot_songs

,artist,title
0,Bob Dylan,Like a Rolling Stone
1,The Rolling Stones,(I Can't Get No) Satisfaction
2,The Beach Boys,Good Vibrations
3,Nirvana,Smells Like Teen Spirit
4,Aretha Franklin,Respect
...,...,...
2993,Black Box Recorder,The Facts of Life
2994,Bob Dylan,Just Like Tom Thumb's Blues
2995,Sebadoh,On Fire
2996,Phoenix,Long Distance Call


In [5]:
hot_songs.drop(columns = 'Unnamed: 0', inplace = True)
hot_songs

,artist,title
0,Brenda Lee,Rockin' Around The Christmas Tree
1,Mariah Carey,All I Want For Christmas Is You
2,Bobby Helms,Jingle Bell Rock
3,Wham!,Last Christmas
4,Burl Ives,A Holly Jolly Christmas
...,...,...
95,Grupo Frontera & Grupo Firme,El Amor de Su Vida
96,Jung Kook,Standing Next To You
97,Morgan Wallen Featuring Eric Church,Man Made A Bar
98,Calle 24 x Chino Pacas x Fuerza Regida,Que Onda


## defining functions

In [6]:
import sys
from config import *
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=Client_ID, client_secret=Client_Secret))

In [7]:
def search_song(title, artist, limit=1):
    search_query = f"track:{title} artist:{artist}"
    id = sp.search(q=search_query, limit=limit)['tracks']['items'][0]['id']
    return id

In [8]:
def add_id(df):
    chunks = 50
    list_of_ids = []

    for i in range(0, len(df), chunks):
        chunk = df.iloc[i:i+chunks]
        print("Collecting IDs for chunk",int(i/chunks))

        for index, row in chunk.iterrows():
            title = row["title"]
            artist = row["artist"]
            try:
                id = search_song(title, artist,1)
                list_of_ids.append(id)
            except:
                print("Song not found!")
                list_of_ids.append("")
        sleep(30)
        print('sleep')
    df["id"] = list_of_ids
    return df

In [9]:
def get_audio_features(list_of_ids):
    chunk_size = 50
    feature_df = pd.DataFrame()
    
    for i in range(0, len(list_of_ids), chunk_size):
        chunk = list_of_ids[i:i+chunk_size]
        print("Collecting audio features for chunk",int(i/chunk_size))
        
        try:
            my_dict = sp.audio_features(list_of_ids[i:i+chunk_size])
            df = pd.DataFrame(my_dict)
            feature_df = pd.concat([feature_df,df], ignore_index=True)
        except:
            print("Error retrieving features")
                
        print("Sleeping")
        sleep(30)
    
    
    return feature_df

In [10]:
def add_audio_features(df:pd.DataFrame, audio_features_df:pd.DataFrame):

    full_df = df.merge(audio_features_df, how="left", on='id')

    return full_df

In [11]:
def drop_empty_rows(df:pd.DataFrame, col:str):
    
    list_of_row_index = df[df[col] == ""].index.tolist()
    df = df.drop(list_of_row_index,axis=0)
    return df

In [12]:
def add_songs_features_run_all(df):   
    df = add_id(df)
    df = drop_empty_rows(df,'id')
    list_of_ids = df["id"].tolist()
    features = get_audio_features(list_of_ids)
    df = add_audio_features(df, features)
    return df

## get the audio features from the hot songs

In [13]:
hot_songs = add_id(hot_songs)
hot_songs = drop_empty_rows(hot_songs,'id')
list_of_ids = hot_songs["id"].tolist()
#features = get_audio_features(list_of_ids)
#hot_songs = add_audio_features(hot_songs, features)

Song not found!
Song not found!
Song not found!
Song not found!
Song not found!
sleep
Song not found!
Song not found!
Song not found!
Song not found!
Song not found!
Song not found!
Song not found!
Song not found!
Song not found!
Song not found!
Song not found!
sleep


In [15]:
hot_songs

,artist,title,id
0,Brenda Lee,Rockin' Around The Christmas Tree,2EjXfH91m7f8HiJN1yQg97
1,Mariah Carey,All I Want For Christmas Is You,0bYg9bo50gSsH3LtXe2SQn
2,Bobby Helms,Jingle Bell Rock,7vQbuQcyTflfCIOu3Uzzya
3,Wham!,Last Christmas,2FRnf9qhLbvw8fu4IBXx78
4,Burl Ives,A Holly Jolly Christmas,77khP2fIVhSW23NwxrRluh
...,...,...,...
93,Sabrina Carpenter,Feather,2Zo1PcszsT9WQ0ANntJbID
94,Olivia Rodrigo,Can't Catch Me Now,56xHMIfQPoe0prrSi3BGhf
95,Grupo Frontera & Grupo Firme,El Amor de Su Vida,0O3U5iwTbiXCREMkvotJuN
96,Jung Kook,Standing Next To You,2KslE17cAJNHTsI2MI0jb2


In [16]:
my_dict = sp.audio_features(list_of_ids[0:50])
pd.DataFrame(my_dict)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.589,0.472,8,-8.749,1,0.0502,0.61400,0.000000,0.5050,0.898,67.196,audio_features,2EjXfH91m7f8HiJN1yQg97,spotify:track:2EjXfH91m7f8HiJN1yQg97,https://api.spotify.com/v1/tracks/2EjXfH91m7f8...,https://api.spotify.com/v1/audio-analysis/2EjX...,126267,4
1,0.336,0.627,7,-7.463,1,0.0384,0.16400,0.000000,0.0708,0.350,150.273,audio_features,0bYg9bo50gSsH3LtXe2SQn,spotify:track:0bYg9bo50gSsH3LtXe2SQn,https://api.spotify.com/v1/tracks/0bYg9bo50gSs...,https://api.spotify.com/v1/audio-analysis/0bYg...,241107,4
2,0.754,0.424,2,-8.463,1,0.0363,0.64300,0.000000,0.0652,0.806,119.705,audio_features,7vQbuQcyTflfCIOu3Uzzya,spotify:track:7vQbuQcyTflfCIOu3Uzzya,https://api.spotify.com/v1/tracks/7vQbuQcyTflf...,https://api.spotify.com/v1/audio-analysis/7vQb...,130973,4
3,0.735,0.478,2,-12.472,1,0.0293,0.18900,0.000002,0.3550,0.947,107.682,audio_features,2FRnf9qhLbvw8fu4IBXx78,spotify:track:2FRnf9qhLbvw8fu4IBXx78,https://api.spotify.com/v1/tracks/2FRnf9qhLbvw...,https://api.spotify.com/v1/audio-analysis/2FRn...,262960,4
4,0.683,0.375,0,-13.056,1,0.0303,0.57900,0.000000,0.0760,0.888,140.467,audio_features,77khP2fIVhSW23NwxrRluh,spotify:track:77khP2fIVhSW23NwxrRluh,https://api.spotify.com/v1/tracks/77khP2fIVhSW...,https://api.spotify.com/v1/audio-analysis/77kh...,135533,4
5,0.294,0.885,4,-7.209,0,0.1060,0.56100,0.001330,0.0990,0.401,112.715,audio_features,3gBhiIj76UJQ7jlrUR4qjc,spotify:track:3gBhiIj76UJQ7jlrUR4qjc,https://api.spotify.com/v1/tracks/3gBhiIj76UJQ...,https://api.spotify.com/v1/audio-analysis/3gBh...,248160,3
6,0.451,0.240,1,-14.014,1,0.0351,0.91200,0.000002,0.1750,0.701,134.009,audio_features,2uFaJJtFpPDc5Pa95XzTvg,spotify:track:2uFaJJtFpPDc5Pa95XzTvg,https://api.spotify.com/v1/tracks/2uFaJJtFpPDc...,https://api.spotify.com/v1/audio-analysis/2uFa...,117147,4
7,0.513,0.831,2,-9.004,1,0.0383,0.55000,0.000000,0.3360,0.963,148.837,audio_features,0oPdaY4dXtc3ZsaG17V972,spotify:track:0oPdaY4dXtc3ZsaG17V972,https://api.spotify.com/v1/tracks/0oPdaY4dXtc3...,https://api.spotify.com/v1/audio-analysis/0oPd...,182067,4
8,0.943,0.558,2,-4.911,1,0.0568,0.00260,0.000002,0.0937,0.606,104.983,audio_features,4xhsWYTOGcal8zt0J161CU,spotify:track:4xhsWYTOGcal8zt0J161CU,https://api.spotify.com/v1/tracks/4xhsWYTOGcal...,https://api.spotify.com/v1/audio-analysis/4xhs...,138411,4
9,0.529,0.772,2,-7.013,1,0.0287,0.40300,0.000002,0.3160,0.853,91.751,audio_features,5ASM6Qjiav2xPe7gRkQMsQ,spotify:track:5ASM6Qjiav2xPe7gRkQMsQ,https://api.spotify.com/v1/tracks/5ASM6Qjiav2x...,https://api.spotify.com/v1/audio-analysis/5ASM...,181267,4


In [17]:
hot_songs_audio_features = add_songs_features_run_all(hot_songs)

sleep


KeyboardInterrupt: 

In [ ]:
hot_songs_audio_features

## get the audio features from the hot not songs

In [ ]:
not_hot_songs_audio_features = add_songs_features_run_all(not_hot_songs)

In [ ]:
not_hot_songs_audio_features

## export csv

In [ ]:
not_hot_songs_audio_features.to_csv("not_hot_songs_audio_features.csv")

In [ ]:
hot_songs_audio_features.to_csv("hot_songs_audio_features.csv")